# ABB IRB-140-6-0-8

In [ ]:
import numpy as np
import sympy as sp
import plotly.graph_objects as go

# -----------------------------
# Define symbolic variables and DH parameters for ABB IRB-140-6-0-8
# -----------------------------
theta1, theta2, theta3, theta4, theta5, theta6 = sp.symbols('theta1 theta2 theta3 theta4 theta5 theta6', real=True)
theta = [theta1, theta2, theta3, theta4, theta5, theta6]

# ABB IRB-140 DH parameters: [a, alpha, d, theta]
# Note: angles are in radians and lengths in meters.
DH_params = [
    [0,       -sp.pi/2,  0.450, theta1],    # Joint 1 (base rotation)
    [0.400,   0,         0,     theta2],    # Joint 2 (shoulder)
    [0.100,   -sp.pi/2,  0,     theta3],    # Joint 3 (elbow)
    [0,       sp.pi/2,   0.300, theta4],    # Joint 4 (wrist roll)
    [0,       -sp.pi/2,  0,     theta5],    # Joint 5 (wrist pitch)
    [0,       0,         0.080, theta6]     # Joint 6 (wrist yaw/tool offset)
]

# -----------------------------
# Function to compute the DH transformation matrix
# -----------------------------
def dh_transform(a, alpha, d, theta):
    return sp.Matrix([
        [sp.cos(theta), -sp.sin(theta)*sp.cos(alpha),  sp.sin(theta)*sp.sin(alpha), a*sp.cos(theta)],
        [sp.sin(theta),  sp.cos(theta)*sp.cos(alpha), -sp.cos(theta)*sp.sin(alpha), a*sp.sin(theta)],
        [0,              sp.sin(alpha),                sp.cos(alpha),               d],
        [0,              0,                            0,                           1]
    ])

# -----------------------------
# Compute Forward Kinematics
# -----------------------------
T = sp.eye(4)
for params in DH_params:
    T *= dh_transform(*params)

# Extract end-effector position (x, y, z)
x, y, z = T[:3, 3]

# Compute the Jacobian matrix (partial derivatives of [x, y, z] with respect to joint angles)
J = sp.simplify(sp.Matrix([x, y, z]).jacobian(theta))

# Compute the Manipulability Index (sqrt(det(J * J^T)))
Manipulability_Index = sp.sqrt(sp.det(J * J.T))

# -----------------------------
# Generate Workspace Data
# -----------------------------
# We sample three joint angles (theta1, theta2, theta3) uniformly over [-pi, pi]
# while holding the remaining joints (theta4, theta5, theta6) at 0.
num_samples = 20
theta_samples = np.linspace(-np.pi, np.pi, num_samples)

# Lists to store workspace data and performance metrics
X, Y, Z, M, CI = [], [], [], [], []

for t1 in theta_samples:
    for t2 in theta_samples:
        for t3 in theta_samples:
            # Evaluate forward kinematics with theta4=theta5=theta6=0.
            T_eval = T.subs({theta1: t1, theta2: t2, theta3: t3,
                             theta4: 0, theta5: 0, theta6: 0})
            x_val, y_val, z_val = [float(val.evalf()) for val in T_eval[:3, 3]]

            # Evaluate the Jacobian numerically for the same configuration.
            J_eval = J.subs({theta1: t1, theta2: t2, theta3: t3,
                             theta4: 0, theta5: 0, theta6: 0})
            J_num = np.array(J_eval).astype(np.float64)

            # Calculate Manipulability Index.
            try:
                M_val = np.sqrt(np.linalg.det(J_num @ J_num.T))
            except np.linalg.LinAlgError:
                M_val = 0
            # Condition index defined as 1 / cond(J); near singularity CI->0.
            try:
                CI_val = 1 / np.linalg.cond(J_num)
            except np.linalg.LinAlgError:
                CI_val = 0

            X.append(x_val)
            Y.append(y_val)
            Z.append(z_val)
            M.append(M_val)
            CI.append(CI_val)

# -----------------------------
# Convert workspace coordinates from meters to millimeters
# -----------------------------
X_mm = np.array(X) * 1000
Y_mm = np.array(Y) * 1000
Z_mm = np.array(Z) * 1000

# -----------------------------
# Define common layout settings with 100mm grid divisions
# -----------------------------
scene_layout = dict(
    xaxis=dict(title='X (mm)', dtick=100),
    yaxis=dict(title='Y (mm)', dtick=100),
    zaxis=dict(title='Z (mm)', dtick=100)
)

# -----------------------------
# 1. Workspace Only (Clear Visualization)
# -----------------------------
fig_workspace = go.Figure(data=[
    go.Scatter3d(
        x=X_mm, y=Y_mm, z=Z_mm,
        mode='markers',
        marker=dict(
            size=3,
            color='blue',
            opacity=0.6
        )
    )
])
fig_workspace.update_layout(
    title='ABB IRB-140 Workspace (mm)',
    scene=scene_layout
)
fig_workspace.show()

# -----------------------------
# 2. Singular Configurations (Using Condition Index Threshold)
# -----------------------------
# For near-singular configurations, we use a threshold on the condition index.
# (Since CI = 1/cond(J), values near 0 indicate near singularity.)
ci_threshold = 0.05  # Adjust threshold as needed

# Filter workspace points based on condition index.
X_singular = [X_mm[i] for i, ci_val in enumerate(CI) if ci_val < ci_threshold]
Y_singular = [Y_mm[i] for i, ci_val in enumerate(CI) if ci_val < ci_threshold]
Z_singular = [Z_mm[i] for i, ci_val in enumerate(CI) if ci_val < ci_threshold]

fig_singular = go.Figure(data=[
    go.Scatter3d(
        x=X_singular, y=Y_singular, z=Z_singular,
        mode='markers',
        marker=dict(
            size=3,
            color='red',
            opacity=0.6
        )
    )
])
fig_singular.update_layout(
    title='ABB IRB-140 Singular Configurations (mm)',
    scene=scene_layout
)
fig_singular.show()

# -----------------------------
# 3. Condition Index Plot (Using Plasma colorscale)
# -----------------------------
fig_condition = go.Figure(data=[
    go.Scatter3d(
        x=X_mm, y=Y_mm, z=Z_mm,
        mode='markers',
        marker=dict(
            size=3,
            color=CI,  # condition index values
            colorscale='Plasma',
            opacity=0.6,
            colorbar=dict(title='Condition Index')
        )
    )
])
fig_condition.update_layout(
    title='ABB IRB-140 - Workspace & Condition Index (mm)',
    scene=scene_layout
)
fig_condition.show()

# -----------------------------
# 4. Manipulability Index Plot (Using Viridis colorscale)
# -----------------------------
fig_manipulability = go.Figure(data=[
    go.Scatter3d(
        x=X_mm, y=Y_mm, z=Z_mm,
        mode='markers',
        marker=dict(
            size=3,
            color=M,  # manipulability index values
            colorscale='Viridis',
            opacity=0.6,
            colorbar=dict(title='Manipulability Index')
        )
    )
])
fig_manipulability.update_layout(
    title='ABB IRB-140 - Workspace & Manipulability Index (mm)',
    scene=scene_layout
)
fig_manipulability.show()
